In [ ]:
!pip install datasets
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# Charger le fichier JSON
with open('squad_arabe_enrichi_cleaned.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extraire les questions et les réponses
questions = []
answers = []
contexts = []  # Contexte des paragraphes pour la tâche de question-réponse

for item in data['data']:
    for paragraph in item['paragraphs']:
        context = paragraph['context']  # Contexte
        for qa in paragraph['qas']:
            question = qa['question']  # Question
            answer_text = " ".join([answer['text'] for answer in qa['answers']])  # Réponse
            questions.append(question)
            answers.append(answer_text)
            contexts.append(context)

# Créer un dataset compatible Hugging Face
dataset = Dataset.from_dict({
    'question': questions,
    'answer': answers,
    'context': contexts
})

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Charger le modèle SambaLingo-Arabic-Chat
model_name = "sambanovasystems/SambaLingo-Arabic-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

# Tokenization des données
def preprocess_function(examples):
    return tokenizer(
        examples['question'], examples['context'], truncation=True, padding=True, max_length=512
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/986k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/780 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [ ]:
# Appliquer la tokenisation
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',          # Répertoire de sortie
    evaluation_strategy="epoch",     # Evaluation par époque
    learning_rate=2e-5,              # Taux d'apprentissage
    per_device_train_batch_size=16,  # Taille du batch d'entraînement
    per_device_eval_batch_size=64,   # Taille du batch d'évaluation
    num_train_epochs=3,              # Nombre d'époques
    weight_decay=0.01,               # Décroissance du poids
)

# Définir le Trainer
trainer = Trainer(
    model=model,                       # Le modèle à fine-tuner
    args=training_args,                # Les arguments d'entraînement
    train_dataset=tokenized_datasets,  # Le dataset d'entraînement
    eval_dataset=tokenized_datasets    # Le dataset d'évaluation (peut être différent)
)

# Fine-tuning du modèle
trainer.train()

# Sauvegarder le modèle fine-tuné
trainer.save_model('./fine_tuned_sambalingo_arabic_chat_model')

Map:   0%|          | 0/145 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


OutOfMemoryError: CUDA out of memory. Tried to allocate 22.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 21.06 MiB is free. Process 9043 has 14.72 GiB memory in use. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 186.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)